In [1]:
# All as One: Gaussian NB
# Treat All Participants as One
# Try with 10% ... 90% for Training (several iterations for each)

In [2]:
use_key_features = True
shuffle_times = 100
iterations = 10

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

# 1 -- Load Data

In [4]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [5]:
data = [data5, data7, data9, data11, data12, data17, data18]
all_data = pd.concat(data, ignore_index=True, sort=True)

In [6]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

all_data = all_data.drop(columns=remove)

In [7]:
# Remove rows where engagment NaN
all_data = all_data[np.isfinite(all_data['engagement'])]

# Remove rows where engagment is -1
all_data = all_data[all_data['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [8]:
# Main Columns
basic_cols = []
for i in all_data.columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [9]:
# Open Face Columns

of_cols = []
for i in all_data.columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [10]:
# Audio Columns

a_cols = []
for i in all_data.columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [11]:
# ROS Columns

ros_cols = []
for i in all_data.columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [12]:
# For Window Only:
non_window_features = []
window_features = []
for i in all_data.columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [13]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [14]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [15]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

all_data = all_data[features_to_keep]

In [16]:
# All as One Participant: Shuffle Data! 

for i in range(shuffle_times):
    all_data.reindex(np.random.permutation(all_data.index))
all_data = all_data.reset_index(drop=True)

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [17]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [18]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing 

# split_size: how much data for training
def split(split_size):
    y_data = all_data['engagement'].copy()
    X_data = all_data.drop(columns=['engagement', 'session_num', 'participant'], axis=1).copy()
        
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=1-split_size, shuffle=True)

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_test, y_test

In [19]:
# MODEL HERE

# Inputs: X_train, y_train, X_test, y_test, hp, isVerbose
# hp = Hyperparameter Dictionary, isVerbose = whether to return all accuracy metrics

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.naive_bayes import GaussianNB

# Hyperparameter Combinations to Try:  
hp_names = ['var_smoothing']
hp_list = [[1e-12,1e-9,1e-8,1e-6]]

def model(X_train, y_train, X_test, y_test, hp, isVerbose):
    # Model Here: must create pred & scores arrays
    clf = GaussianNB(var_smoothing=hp['var_smoothing'])
    clf.fit(X_train.values, y_train.values)

    pred = clf.predict(X_test.values)
    scores = clf.predict_proba(X_test.values)[:,1]
        
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    if isVerbose:
        try:
            auc = roc_auc_score(y_test, scores)
        except:
            auc = np.nan
        results['auc'] = auc
    
        precision = metrics.precision_score(y_test, pred, average=None)
        recall = metrics.recall_score(y_test, pred, average=None)
        f1 = metrics.f1_score(y_test, pred, average=None)

        results['precision_0'], results['precision_1'] = precision[0], precision[1]
        results['recall_0'], results['recall_1'] = recall[0], recall[1]
        results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [20]:
all_results = []
for perc in [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    print('Training Percentage:', perc)
    for n in range(iterations):
        print()
        new_results = {}
        
        # Shuffle Data For This Iteration
        for i in range(shuffle_times):
            all_data.reindex(np.random.permutation(all_data.index))        
        all_data = all_data.reset_index(drop=True)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_test, y_test = split(perc)
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)

        # Set Up Cross Validation Groups
        div = int(len(X_train)/3)
        X_cv1 = X_train.loc[:div]
        X_cv2 = X_train.loc[div+1:div*2]
        X_cv3 = X_train.loc[(div*2)+1:]
        
        y_cv1 = y_train.loc[:div]
        y_cv2 = y_train.loc[div+1:div*2]
        y_cv3 = y_train.loc[(div*2)+1:]
        
        cv1_X_train = X_cv1.append(X_cv2)
        cv1_X_test = X_cv3
        cv1_y_train = y_cv1.append(y_cv2)
        cv1_y_test = y_cv3
        
        cv2_X_train = X_cv1.append(X_cv3)
        cv2_X_test = X_cv2
        cv2_y_train = y_cv1.append(y_cv3)
        cv2_y_test = y_cv2

        cv3_X_train = X_cv3.append(X_cv2)
        cv3_X_test = X_cv1
        cv3_y_train = y_cv3.append(y_cv2)
        cv3_y_test = y_cv1

        # Hyperparameter Combinations        
        hp_combos = list(itertools.product(*hp_list))
        best, best_acc = None, 0
        for h in hp_combos:
            hp = {}
            for i,v in enumerate(h):
                hp[hp_names[i]] = v
            
            # Cross-Validation
            res1 = model(cv1_X_train, cv1_y_train, cv1_X_test, cv1_y_test, hp, False)
            res2 = model(cv2_X_train, cv2_y_train, cv2_X_test, cv2_y_test, hp, False)
            res3 = model(cv3_X_train, cv3_y_train, cv3_X_test, cv3_y_test, hp, False)
            
            acc = np.mean([res1['accuracy'], res2['accuracy'], res3['accuracy']])
            print(hp, acc)
            if acc > best_acc:
                best = hp
                best_acc = acc
        
        # Evaluate on Test Using Best Hyperaparemeters  
        print('Best HP', best)
        results = model(X_train, y_train, X_test, y_test, best, True)
        print(results)
        results['train_percentage'] = perc
        results['iteration'] = n
        results['hyperparameters'] = json.dumps(best)
        all_results.append(results)

Training Percentage: 0.1

{'var_smoothing': 1e-12} 0.7531417096878225
{'var_smoothing': 1e-09} 0.7531417096878225
{'var_smoothing': 1e-08} 0.7531417096878225
{'var_smoothing': 1e-06} 0.7531417096878225
Best HP {'var_smoothing': 1e-12}
{'precision_1': 0.816396749944727, 'recall_0': 0.6611669301779977, 'f1_1': 0.8077243830747146, 'auc': 0.7931712786817495, 'recall_1': 0.7992343280755391, 'accuracy': 0.7513788714467543, 'f1_0': 0.6483208721962441, 'precision_0': 0.6359644819466248}

{'var_smoothing': 1e-12} 0.7472572019779694
{'var_smoothing': 1e-09} 0.7472572019779694
{'var_smoothing': 1e-08} 0.7472572019779694
{'var_smoothing': 1e-06} 0.7472572019779694
Best HP {'var_smoothing': 1e-12}
{'precision_1': 0.817393112050447, 'recall_0': 0.6639289904606438, 'f1_1': 0.8074614094534004, 'auc': 0.7932401547907073, 'recall_1': 0.7977681590693899, 'accuracy': 0.7513877103662848, 'f1_0': 0.6492324192200731, 'precision_0': 0.6351723969644469}

{'var_smoothing': 1e-12} 0.7581541936093253
{'var_smooth

{'var_smoothing': 1e-12} 0.7502482269200602
{'var_smoothing': 1e-09} 0.7502482269200602
{'var_smoothing': 1e-08} 0.7502482269200602
{'var_smoothing': 1e-06} 0.7502482269200602
Best HP {'var_smoothing': 1e-12}
{'precision_1': 0.8182115339715152, 'recall_0': 0.6646856158484066, 'f1_1': 0.8088321280196953, 'auc': 0.7949304301078721, 'recall_1': 0.7996653228873507, 'accuracy': 0.7529160244617908, 'f1_0': 0.6507659873506676, 'precision_0': 0.6374174008810573}

{'var_smoothing': 1e-12} 0.7463106401312891
{'var_smoothing': 1e-09} 0.7463106401312891
{'var_smoothing': 1e-08} 0.7463106401312891
{'var_smoothing': 1e-06} 0.7463504173787036
Best HP {'var_smoothing': 1e-06}
{'precision_1': 0.8148806579272817, 'recall_0': 0.6557820000574894, 'f1_1': 0.8080849889624724, 'auc': 0.7915315099131974, 'recall_1': 0.8014017270737047, 'accuracy': 0.7510266991498036, 'f1_0': 0.6456853366541194, 'precision_0': 0.6358948630041531}
Training Percentage: 0.3

{'var_smoothing': 1e-12} 0.7522739015291579
{'var_smoot

{'var_smoothing': 1e-12} 0.7465893353879571
{'var_smoothing': 1e-09} 0.7465893353879571
{'var_smoothing': 1e-08} 0.7465893353879571
{'var_smoothing': 1e-06} 0.746629110262169
Best HP {'var_smoothing': 1e-06}
{'precision_1': 0.8143995400032857, 'recall_0': 0.6535306294564134, 'f1_1': 0.8090662423240915, 'auc': 0.7904248260151556, 'recall_1': 0.803802343021606, 'accuracy': 0.7518296563428086, 'f1_0': 0.6455863975461051, 'precision_0': 0.637832984136486}

{'var_smoothing': 1e-12} 0.7506463808637859
{'var_smoothing': 1e-09} 0.7506463808637859
{'var_smoothing': 1e-08} 0.7506463808637859
{'var_smoothing': 1e-06} 0.7506463808637859
Best HP {'var_smoothing': 1e-12}
{'precision_1': 0.8135603715170279, 'recall_0': 0.6526971432965512, 'f1_1': 0.8055382414550658, 'auc': 0.7873038920007092, 'recall_1': 0.7976727714256805, 'accuracy': 0.7476797836232499, 'f1_0': 0.6408093161957609, 'precision_0': 0.62934677837918}

{'var_smoothing': 1e-12} 0.7513821839984732
{'var_smoothing': 1e-09} 0.75138218399847

{'var_smoothing': 1e-12} 0.7482333125634458
{'var_smoothing': 1e-09} 0.7482333125634458
{'var_smoothing': 1e-08} 0.7482333125634458
{'var_smoothing': 1e-06} 0.7482465716459172
Best HP {'var_smoothing': 1e-06}
{'precision_1': 0.814068794546018, 'recall_0': 0.6577881708777733, 'f1_1': 0.8080590587511536, 'auc': 0.7918048365067747, 'recall_1': 0.8021374045801527, 'accuracy': 0.7518047849173677, 'f1_0': 0.6489056433916615, 'precision_0': 0.6402598123577417}

{'var_smoothing': 1e-12} 0.747755966081589
{'var_smoothing': 1e-09} 0.747755966081589
{'var_smoothing': 1e-08} 0.747755966081589
{'var_smoothing': 1e-06} 0.747755966081589
Best HP {'var_smoothing': 1e-12}
{'precision_1': 0.815104328869511, 'recall_0': 0.6592826398852224, 'f1_1': 0.8057197389484053, 'auc': 0.7894074567900675, 'recall_1': 0.7965487856838517, 'accuracy': 0.7489807688483185, 'f1_0': 0.645429518512276, 'precision_0': 0.6321465910966819}

{'var_smoothing': 1e-12} 0.7493336655794449
{'var_smoothing': 1e-09} 0.7493336655794449

{'var_smoothing': 1e-12} 0.7491746415476346
{'var_smoothing': 1e-09} 0.7491746415476346
{'var_smoothing': 1e-08} 0.7491746415476346
{'var_smoothing': 1e-06} 0.7492044735932181
Best HP {'var_smoothing': 1e-06}
{'precision_1': 0.8061818632309217, 'recall_0': 0.6480314960629922, 'f1_1': 0.8034942585504383, 'auc': 0.7903261676956884, 'recall_1': 0.8008245139059808, 'accuracy': 0.7467981863018057, 'f1_0': 0.6441189624329159, 'precision_0': 0.6402533896421427}

{'var_smoothing': 1e-12} 0.7479415507363195
{'var_smoothing': 1e-09} 0.7479415507363195
{'var_smoothing': 1e-08} 0.7479415507363195
{'var_smoothing': 1e-06} 0.747941551032961
Best HP {'var_smoothing': 1e-06}
{'precision_1': 0.8184920881166615, 'recall_0': 0.6598837209302325, 'f1_1': 0.8077900603239735, 'auc': 0.7879280506561172, 'recall_1': 0.7973642848506831, 'accuracy': 0.7503380797072627, 'f1_0': 0.6438985647018778, 'precision_0': 0.6286695469148111}

{'var_smoothing': 1e-12} 0.7506562817164791
{'var_smoothing': 1e-09} 0.7506562817

In [21]:
results = pd.DataFrame(columns=['train_percentage', 'iteration', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'hyperparameters'])
results = results.append(all_results, ignore_index=True, sort=True)

In [22]:
results.head()

,accuracy,auc,f1_0,f1_1,hyperparameters,iteration,precision_0,precision_1,recall_0,recall_1,train_percentage
0,0.751379,0.793171,0.648321,0.807724,"{""var_smoothing"": 1e-12}",0,0.635964,0.816397,0.661167,0.799234,0.1
1,0.751388,0.793240,0.649232,0.807461,"{""var_smoothing"": 1e-12}",1,0.635172,0.817393,0.663929,0.797768,0.1
2,0.749823,0.792396,0.646306,0.806466,"{""var_smoothing"": 1e-12}",2,0.633265,0.815657,0.659896,0.797479,0.1
3,0.747233,0.788131,0.642309,0.804563,"{""var_smoothing"": 1e-12}",3,0.629746,0.813429,0.655385,0.795887,0.1
4,0.751087,0.792438,0.648712,0.807258,"{""var_smoothing"": 1e-06}",4,0.633330,0.818160,0.664860,0.796642,0.1


In [23]:
results.to_csv('results.csv', index=False)